In [1]:
import pandas as pd

data = pd.read_csv("menu.csv")
data.head()

C:\Users\farha\AppData\Local\Temp\ipykernel_22020\500525866.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,Name,Type,Quantity,Ingredients,Price
0,Burger,Patty,1 Nos,"Burger bun, patty (beef, chicken, or veg), let...",100
1,Sandwich,Layered,1 Nos,"Bread slices, filling (vegetables, cheese, mea...",60
2,Pizza,Cheesy,1 medium,"Pizza dough, pizza sauce, cheese, toppings (pe...",300
3,French Fries,Crispy,250 grams,"Potatoes, oil, salt",80
4,Hot Dogs,Snappier,2 Nos,"Hotdog buns, sausages (chicken, pork, beef), m...",120


In [2]:
from translate import Translator

location = "Coimbatore"
language = "Tamil"
if language == "Tamil":
    translator = Translator(to_lang='ta')
elif language == "Hindi":
    translator = Translator(to_lang='hindi')


In [3]:
dataLst = []

for index, row in data.iterrows():
    dataLst.append([row['Name'], row['Type'], row['Quantity'], row['Ingredients'], row['Price']])
print(dataLst)

[['Burger', 'Patty', '1 Nos', 'Burger bun, patty (beef, chicken, or veg), lettuce, tomato, onion, cheese, sauce (mayonnaise, ketchup), pickles', 100], ['Sandwich', 'Layered', '1 Nos', 'Bread slices, filling (vegetables, cheese, meat), butter, sauce (mayonnaise, mustard)', 60], ['Pizza', 'Cheesy', '1 medium', 'Pizza dough, pizza sauce, cheese, toppings (pepperoni, vegetables, etc.), herbs (basil, oregano), olive oil', 300], ['French Fries', 'Crispy', '250 grams', 'Potatoes, oil, salt', 80], ['Hot Dogs', 'Snappier', '2 Nos', 'Hotdog buns, sausages (chicken, pork, beef), mustard, ketchup, onions, relish', 120]]


In [4]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
load_dotenv()

genai.configure(api_key=os.getenv('API_KEY'))

c:\Users\farha\anaconda3\envs\mistralenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import PIL.Image

img = PIL.Image.open("pic.jpeg")

imgModel = genai.GenerativeModel("gemini-pro-vision")
response = imgModel.generate_content(["What is the name of the dish\nNote:ouput should be in one word format - \"dish\"", img])
imgDish = response.text

response = imgModel.generate_content(["what all ingredients are there in this food image, give me the ingredients in list. always give in english language.\nNote:ouput should be in the format - \"[Burger bun, patty (beef, chicken, or veg), lettuce, tomato, onion, cheese, sauce (mayonnaise, ketchup), pickles]\"", img])
imgIngredients = response.text

response = imgModel.generate_content(["I have given you a image of a cooked food, give me 1 min of reading content in english Describe the pictured dish in 30 words not more than that. Analyze the cooking style based on visual clues like browning, charring, or visible moisture. Based on these indicators, predict the flavor profile. Mention potential textures and dominant tastes (e.g., smoky, savory, sweet). the response should not be more than 30 words", img])
imgDiscription = response.text


print(f"{imgDish}\n{imgIngredients}\n{imgDiscription}")

 Pizza
 [Pizza base, tomato sauce, cheese (mozzarella, cheddar, parmesan), basil leaves]
 This is a cheese pizza. The pizza has a thin crust and is topped with melted cheese and basil. The crust is golden brown and the cheese is bubbly and slightly browned. The pizza has a crispy texture and a slightly smoky flavor. The dominant tastes are cheese and tomato sauce.


In [9]:
generation_config = {"temperature": 0.9, "top_p": 1, "top_k": 1, "max_output_tokens": 2048}

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"}
]

txtModel = genai.GenerativeModel(model_name="gemini-1.0-pro",
generation_config=generation_config,
safety_settings=safety_settings)

convo = txtModel.start_chat(history=[])


In [10]:
dishes = data.iloc[:, 0]
#print(dishes.to_numpy())

convo.send_message(f"What is the overall cusine for {dishes.to_numpy} from (american, indian, chineese)\nNote:ouput should be in one word format - \"cusine\"")
cusine = convo.last.text
print(cusine)


american


In [11]:
convo.send_message(f"Based on the cusine give me two more dishes apart from {dishes}\nNote:ouput should be in the format - \"dish1, dish2\"")
extraDishes = convo.last.text
print(extraDishes)

Macaroni, Pancakes


In [12]:
convo = txtModel.start_chat(history=[])

convo.send_message(f"Give me the index of {imgDish} in the list: {dishes}\nNote:ouput should be in one number format - \"index\"")
dishIndex = convo.last.text
print(dishIndex)

2


In [14]:
#imgIngredients = "[Pizza base, tomato sauce, cheese, basil]"
imgIngredients = imgIngredients[2:-1]

curDish = dataLst[int(dishIndex)]
print(curDish)
curIngredients = curDish[3] + ", " + imgIngredients
print(curIngredients)

['Pizza', 'Cheesy', '1 medium', 'Pizza dough, pizza sauce, cheese, toppings (pepperoni, vegetables, etc.), herbs (basil, oregano), olive oil', 300]
Pizza dough, pizza sauce, cheese, toppings (pepperoni, vegetables, etc.), herbs (basil, oregano), olive oil, zza base, tomato sauce, cheese (mozzarella, cheddar, parmesan), basil leave


In [15]:
convo = txtModel.start_chat(history=[])

convo.send_message(f"for the dish: \"{curDish[0]}\"give me the nutrients with respect to the ingredients: \"{curIngredients}\", type: \"{curDish[1]}\" and quantity: \"{curDish[2]}\"\nNote:ouput should be in the format - \"[calories:100, carbohydrates:75g, ...]\"")
nutrients = convo.last.text
print(nutrients)

[calories:2500, carbohydrates:400g, protein:150g, fat:100g, sodium:2500mg, fiber:20g]


In [16]:
convo.send_message(f"Is {curDish[4]} rupees for the above dish is (cheap, budget, overpriced) in {location}\nNote:ouput should be in one word format - \"pricing\"")
pricing = convo.last.text
print(pricing)

budget


In [17]:
convo.send_message(f"Is the above dish met the fssai standards?\nIf yes give the reason, if on give me the discrepency\nNote:ouput should be in the format - \"yes/no-reason\"")
fssai = convo.last.text
print(fssai)

yes-The dish contains all of the required nutrients in the correct proportions.


In [18]:
convo.send_message(f"write a review for the above dish in 30 words")
review = convo.last.text
print(review)
print(translator.translate(review))

This cheesy pizza is a delicious and satisfying meal. The crust is crispy and the toppings are generous. The cheese is melted and gooey, and the sauce is flavorful. Overall, this is a great pizza that I would highly recommend.
இந்த சீசி பீட்சா ஒரு சுவையான மற்றும் திருப்திகரமான உணவு. மேலோடு மிருதுவானது மற்றும் மேல்புறங்கள் தாராளமானவை. பாலாடைக்கட்டி உருகுகிறது மற்றும் கூவிக்கொள்கிறது, மேலும் சாஸ் சுவையாக இருக்கிறது. ஒட்டுமொத்தமாக, இது நான் மிகவும் பரிந்துரைக்கும் ஒரு சிறந்த பீட்சா.


In [19]:
convo.send_message(f"give me a rating for the above dish out of 5\nNote:ouput should be in one float format - \"[rating]\"")
rating = convo.last.text
print(rating)

[4.5]


In [20]:
convo = txtModel.start_chat(history=[])
restaurent = "pizza hut"
convo.send_message(f"give me 5 restarents other then {restaurent} that serve {curDish[0]} in {location}\nNote:ouput should be in the format - \"[restaurent1, restaurent2, ...]\"")
otherRestaurents = convo.last.text
print(otherRestaurents)

[Domino's, Mozzarella Pizza Cafe, The Pizza Place, Pizza Corner, Cheese Factory]
